https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/

#### to load elmo to local disk:
#download the model to local so it can be used again and again
$ mkdir module/module_elmo2
# Download the module, and uncompress it to the destination folder. 
$ curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module/module_elmo2

In [1]:
# imports for elmo

import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm  # progress bar
import re
import time
import pickle  # pickle is a file loader?
pd.set_option('display.max_colwidth', 200)

In [2]:
# resumes only for now
# gives df with list of strings (tokenized) as well as lemmatized list of strings

ec2_resumes_path = '/home/ubuntu/NLP_projects/job_recommender_project/data/large_files/lf_cleaned_lemmatized_tokenized_resumes.csv'
resumes = pd.read_csv(ec2_resumes_path)


In [3]:
type(resumes['lemmatized_resume'])

pandas.core.series.Series

In [28]:
# ##### can use the above to turn resumes['lemmatized_resume'] into a list'
# ##### saving and loading w/CSV makes things strings. should have pickled it instead to get list

from ast import literal_eval

# # extract ELMo embeddings from each batch
resumes['lemmatized_resume_list'] = resumes['lemmatized_resume'].apply(literal_eval) 



In [6]:

# resumes['lemmatized _resume'] is pandas.core.series.Series. We need a list
resumes_sentences = resumes['lemmatized_resume'].tolist()


In [16]:
import tensorflow as tf
import tensorflow_hub as hub
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

In [40]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


Using:https://towardsdatascience.com/transfer-learning-using-elmo-embedding-c4a7e415103c
        
key!! saving elmo2 locally        

In [8]:
elmo = hub.Module("/home/ubuntu/module/module_elmo2", trainable=False)

In [ ]:
# embeddings = elmo(
# ["the cat is on the mat", "what are you doing in evening"],
# signature="default",
# as_dict=True)["elmo"]
# with tf.Session() as session:
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#     message_embeddings = session.run(embeddings)

In [ ]:
# message_embeddings.shape

In [34]:
# elmo = hub.Module("/home/ubuntu/module/module_elmo2", trainable=False)
# tokens_input = [["the", "cat", "is", "on", "the", "mat"],
# ["what", "are", "you", "doing", "in", "evening"]]
# tokens_length = [6, 5]
# embeddings = elmo(
# inputs={
# "tokens": tokens_input,
# "sequence_len": tokens_length
# },
# signature="tokens",
# as_dict=True)["elmo"]
# with tf.Session() as session:
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#     message_embeddings2 = session.run(embeddings)

In [35]:
# message_embeddings2.shape

When using in REST API or backend for multiple inputs, rather than initializing sessions for each call, which is a overhead, one efficient way can be:

In [ ]:
# def embed_elmo2(module):
#     with tf.Graph().as_default():
#         sentences = tf.placeholder(tf.string)
#         embed = hub.Module(module)
#         embeddings = embed(sentences)
#         session = tf.train.MonitoredSession()
#     return lambda x: session.run(embeddings, {sentences: x})

# embed_fn = embed_elmo2('/home/ubuntu/module/module_elmo2')
# embed_fn(["i am sambit"]).shape

tds article has X = array of clean text; Y as array of sentiment (0 for negative, 1 for positive)

### now going back to analytics vidhya article 

In [9]:
#### MY ELMO_VECTORS FUNCTION

# def elmo_vectors(x):
#     if type(x) == np.ndarray:
#         print('input is an np.ndarray. using \'to list()\'')
#         embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]    
#     elif type(x) == list:
#         print('input is an list')
#         embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
#     else:
#         print('input is not a np.ndarray or a list')
#         embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

#     with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
#         sess.run(tf.tables_initializer())
#         # return average of ELMo features
#         return sess.run(tf.reduce_mean(embeddings,1))

In [36]:
#### ANALYTICS VIDYHA ELMO_VECTORS_FUNCTION
def elmo_vectors(x, options = run_opts):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [37]:
#### keep batches in list
# list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
resumes_sentences_small = resumes_sentences[0:10]
small_batches = [resumes_sentences_small[i:i+2] for i in range(0,len(resumes_sentences_small),2)]
# batches of 2, total of 10 (out of 1200 resumes)



In [38]:
type(resumes['lemmatized_resume_list'][0])

list

In [39]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(resumes['lemmatized_resume']) for x in small_batches]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[1219,7259,50,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node module_1_apply_default/bilm/embedding_lookup (defined at /home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Mean_2/_231]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[1219,7259,50,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node module_1_apply_default/bilm/embedding_lookup (defined at /home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)


In [ ]:

# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [ ]:
small_resumes_sentences = resumes_sentences[0:100]

In [ ]:
len(small_resumes_sentences)

In [ ]:
type(small_resumes_sentences[0])

In [ ]:
small_resumes_sentences[0]

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [ ]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

In [ ]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [ ]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [ ]:
preds_valid = lreg.predict(xvalid)

In [ ]:
f1_score(yvalid, preds_valid)

In [ ]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [ ]:
# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)

## Other things to try....

# Will try gluon for pre-trained ELMo
https://gluon-nlp.mxnet.io/master/examples/sentence_embedding/elmo_sentence_representation.html?highlight=elmo

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')
# import numpy as np
# import io

# import mxnet as mx
# from mxnet import gluon
# import gluonnlp as nlp
# nlp.utils.check_version('0.7.0')